In [1]:
# como o Kaggle roda o teste sem acesso a internet, precisamos ter anexado todas as dependências necessárias para fazer
# o código functionar.
try:
    import pylibjpeg
except:
    !rm -rf /root/.cache/torch/hub/checkpoints/
    !mkdir -p /root/.cache/torch/hub/checkpoints/
    !pip install /kaggle/input/for-pydicom/dicomsdl-0.109.1-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
    !pip install /kaggle/input/for-pydicom/python_gdcm-3.0.22-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
    !pip install /kaggle/input/for-pydicom/pylibjpeg_libjpeg-1.3.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
    !pip install /kaggle/input/for-pydicom/pylibjpeg-1.4.0-py3-none-any.whl

Processing /kaggle/input/for-pydicom/dicomsdl-0.109.1-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
Processing /kaggle/input/for-pydicom/python_gdcm-3.0.22-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/for-pydicom/pylibjpeg_libjpeg-1.3.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/for-pydicom/pylibjpeg-1.4.0-py3-none-any.whl


In [2]:
import pandas as pd
import numpy as np
import pylibjpeg
import cv2
import os
import gdcm
import pydicom
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [10]:
# configurações iniciais de caminhos dos arquivos no Kaggle e tamanho da imagem convertida (dcm-png).
path = '/kaggle/input/rsna-breast-cancer-detection/'
test_csv = path + 'test.csv'
test_path = path + 'test_images/'
output_path = '/kaggle/working/'
test_output_path = output_path + 'test_images/'
model_path = '/kaggle/input/baseline/baseline_vgg16.h5'
img_size=256

In [4]:
# carrega o CSV do conjunto de testes da avaliação do Kaggle.
test_df = pd.read_csv(test_csv)

In [5]:
# converte a imagem dicom (dcm) para png.
def dicom_to_png(dicom_path):
    # o formato dicom contém uma série de metadados com anotações médicas,
    # mas a imagem fica contida no campo pixel array.
    dicom = pydicom.dcmread(dicom_path)
    img = dicom.pixel_array
    img = (img - img.min()) / (img.max() - img.min())
    
    if dicom.PhotometricInterpretation == 'MONOCHROME1':
        img = 1 - img

    img = cv2.resize(img, (img_size, img_size), interpolation=cv2.INTER_LINEAR)
    img = (img * 255).astype(np.uint8)
    return img

# orquestra a conversão das imagens para salvamento dos pngs na
# pasta de output do kaggle.
def process(dicom_path, image_path):
    image = dicom_to_png(dicom_path)
    os.makedirs(os.path.dirname(image_path), exist_ok=True)
    cv2.imwrite(image_path, image)

In [6]:
# adiciona duas novas colunas no dataframe: uma com o caminho da imagem original e
# outra com o caminho da imagem convertida para png.
test_df['dicom_path'] = test_path + test_df['patient_id'].astype(str) + '/' + test_df['image_id'].astype(str) + '.dcm'
test_df['image_path'] = test_output_path + test_df['patient_id'].astype(str) + '/' + test_df['image_id'].astype(str) + '.png'
test_df

,site_id,patient_id,image_id,laterality,view,age,implant,machine_id,prediction_id,dicom_path,image_path
0,2,10008,736471439,L,MLO,81,0,21,10008_L,/kaggle/input/rsna-breast-cancer-detection/tes...,/kaggle/working/test_images/10008/736471439.png
1,2,10008,1591370361,L,CC,81,0,21,10008_L,/kaggle/input/rsna-breast-cancer-detection/tes...,/kaggle/working/test_images/10008/1591370361.png
2,2,10008,68070693,R,MLO,81,0,21,10008_R,/kaggle/input/rsna-breast-cancer-detection/tes...,/kaggle/working/test_images/10008/68070693.png
3,2,10008,361203119,R,CC,81,0,21,10008_R,/kaggle/input/rsna-breast-cancer-detection/tes...,/kaggle/working/test_images/10008/361203119.png


In [7]:
# infelizmente o Keras não trabalha com imagens dcm diretamente, então o processo de
# conversão para um formato válido (png) é necessário, isso deixa o processo de predição
# extremamente lento - levando em torno de 8 horas.
# tentamos paralelizar este trabalho mas, por ser um processo que, além de exigir CPU
# também exige IO para ler e salvar imagens no storage, o ganho não é tão grande.
Parallel(n_jobs=4, backend='threading')(delayed(process)(dicom_path, image_path) 
                   for dicom_path, image_path in tqdm(zip(test_df['dicom_path'], 
                                                          test_df['image_path'])))

0it [00:00, ?it/s]

[None, None, None, None]

In [12]:
# cria o image data generator apontado para o diretório com as imagens
# pngs geradas acima, no tamanho necessário e usando classificador binário.
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(
    "/kaggle/working/test_images/",
    target_size = (img_size, img_size),
    batch_size = 64,
    class_mode = 'binary')

Found 4 images belonging to 1 classes.


In [13]:
# cria mais duas colunas no dataframe:
#  * cancer: para armazenar a predição do modelo.
#  * prediction_id: o ID do paciente mais a lateralidade (L ou R), esperado pelo avaliador do Kaggle.
test_df['cancer'] = 0
test_df['prediction_id'] = test_df['patient_id'].astype(str) + '_' + test_df['laterality']

# carrega o modelo pré-treinado e, dado as imagens em pngs carregadas no image data generator
# acima, prediz as chances de cancer para cada uma das imagens.
model = load_model(model_path)
pred = model.predict(test_generator)

1/1 [==============================] - 5s 5s/step


In [14]:
# atualiza a coluna "cancer" com os dados obtidos na predição
test_df["cancer"] = pred
test_df

,site_id,patient_id,image_id,laterality,view,age,implant,machine_id,prediction_id,dicom_path,image_path,cancer
0,2,10008,736471439,L,MLO,81,0,21,10008_L,/kaggle/input/rsna-breast-cancer-detection/tes...,/kaggle/working/test_images/10008/736471439.png,0.378988
1,2,10008,1591370361,L,CC,81,0,21,10008_L,/kaggle/input/rsna-breast-cancer-detection/tes...,/kaggle/working/test_images/10008/1591370361.png,0.181860
2,2,10008,68070693,R,MLO,81,0,21,10008_R,/kaggle/input/rsna-breast-cancer-detection/tes...,/kaggle/working/test_images/10008/68070693.png,0.253058
3,2,10008,361203119,R,CC,81,0,21,10008_R,/kaggle/input/rsna-breast-cancer-detection/tes...,/kaggle/working/test_images/10008/361203119.png,0.395397


In [15]:
# o kaggle espera predição por paciente e lateralidade, então agrupamos as colunas e geramos
# um novo dataframe, que será usado para gerar o csv de resultado.
pre_result = test_df[['prediction_id', 'cancer']].groupby("prediction_id").mean().reset_index()
pre_result

,prediction_id,cancer
0,10008_L,0.280424
1,10008_R,0.324228


In [16]:
# como desejamos um resultado binário, colocamos um threshold para normalizar a previsão em 0 ou 1
# e salvamos o csv com o nome e local designado pela competiação, no formato também espeficado:
has_cancer = (pre_result['cancer'] > 0.5).astype(float)
pre_result['cancer'] = has_cancer
pre_result.to_csv('/kaggle/working/submission.csv', index=False)
pre_result

,prediction_id,cancer
0,10008_L,0.0
1,10008_R,0.0
